# Amazone  Android Phone Web Scapper
 * Selemium
 * Beautiful soup

In [112]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

In [113]:
from msedge.selenium_tools import Edge,EdgeOptions

In [114]:
driver=webdriver.Chrome(executable_path="chromedriver.exe")

In [115]:
url = "https://www.amazon.in/"

In [116]:
driver.get(url)

In [117]:
def get_url(search_item):
    template="https://www.amazon.in/s?k={}&crid=1GNY6Q6AHOOKS&sprefix=and%2Caps%2C524&ref=nb_sb_ss_ts-oa-p_1_3"
    search_item=search_item.replace(' ','+')
    return template.format(search_item)

In [118]:
url =get_url("android phone")
print(url)

https://www.amazon.in/s?k=android+phone&crid=1GNY6Q6AHOOKS&sprefix=and%2Caps%2C524&ref=nb_sb_ss_ts-oa-p_1_3


# Extraction the collection


In [119]:
driver.get(url)

In [120]:
soup= BeautifulSoup(driver.page_source,'html.parser')

In [121]:
results =soup.find_all('div',{"data-component-type":"s-search-result"})

In [122]:
len(results)

17

# Prototype  the Record


In [123]:
url="https://www.amazon.in/" +atag.get('href')

In [124]:
item = results[0]

In [125]:
atag=item.h2.a

In [126]:
description=atag.text.strip()

In [127]:
description

'Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery'

In [128]:
price_present=item.find('span','a-price')

In [129]:
price_present

<span class="a-price" data-a-color="price" data-a-size="l"><span class="a-offscreen">₹7,499</span><span aria-hidden="true"><span class="a-price-symbol">₹</span><span class="a-price-whole">7,499</span></span></span>

In [130]:
price=price_present.find('span' ,'a-offscreen').text

In [131]:
price

'₹7,499'

In [132]:
rating=item.i.text

In [133]:
rating

'4.0 out of 5 stars'

In [134]:
review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text

In [135]:
review_count

'27,932'

# gernalize patter

In [136]:
def extract_record(item):
    #description Url and heading
    atag=item.h2.a
    description=atag.text.strip()
    url="https://www.amazon.in/" +atag.get('href')
    #price
    price_present=item.find('span','a-price')
    price=price_present.find('span' ,'a-offscreen').text
    #rating and review
    rating=item.i.text
    review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
    
    results=(description,price,rating,review_count,url)
    
    return results

In [111]:
extract_record

<function __main__.extract_record(item)>

# Use  try execept  for Error handeling

In [141]:
def extract_record(item):
    #description Url and heading
    atag=item.h2.a
    description=atag.text.strip()
    url="https://www.amazon.in/" +atag.get('href')
    try:
        
        #price
        price_present=item.find('span','a-price')
        price=price_present.find('span' ,'a-offscreen').text
    except AttributeError:
        return
    try:
        
        #rating and review
        rating=item.i.text
        review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
    except AttributeError:
        rating=''
        review_count
    results=(description,price,rating,review_count,url)
    
    return results

In [142]:
records=[]
results =soup.find_all('div',{"data-component-type":"s-search-result"})

for item in results:
    record=extract_record(item)
    if record:
        records.append(record)

In [143]:
records[0]

('Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery',
 '₹7,499',
 '4.0 out of 5 stars',
 '27,932',
 'https://www.amazon.in//Redmi-8A-Dual-Blue-Storage/dp/B07X4R63DF/ref=sr_1_1?crid=1GNY6Q6AHOOKS&dchild=1&keywords=android+phone&qid=1605277002&sprefix=and%2Caps%2C524&sr=8-1')

In [146]:
for i in records:
    print(i[1])

₹7,499
₹5,000
₹5,999
₹7,499
₹5,999
₹5,000
₹6,999
₹7,999
₹13,999
₹12,499
₹7,499
₹6,999
₹5,000
₹4,999
₹11,990
₹8,990
₹10,999


# How We track the next page ?
 ### Geting the next page

In [158]:
def get_url(search_item):
    template="https://www.amazon.in/s?k={}&crid=1GNY6Q6AHOOKS&sprefix=and%2Caps%2C524&ref=nb_sb_ss_ts-oa-p_1_3"
    search_item=search_item.replace(' ','+')
    #add query tool
    url =template.format(search_item)
    url+='&page'
    return template.format(search_item)

In [156]:
#all together
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_item):
    template="https://www.amazon.in/s?k={}&crid=1GNY6Q6AHOOKS&sprefix=and%2Caps%2C524&ref=nb_sb_ss_ts-oa-p_1_3"
    search_item=search_item.replace(' ','+')
    #add query tool
    url =template.format(search_item)
    url+='&page{}'
    return url
def extract_record(item):
    #description Url and heading
    atag=item.h2.a
    description=atag.text.strip()
    url="https://www.amazon.in/" +atag.get('href')
    try:
        
        #price
        price_present=item.find('span','a-price')
        price=price_present.find('span' ,'a-offscreen').text
    except AttributeError:
        return
    try:
        
        #rating and review
        rating=item.i.text
        review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
    except AttributeError:
        rating=''
        review_count
    results=(description,price,rating,review_count,url)
    
    return results

def main(search_item):
    driver=webdriver.Chrome(executable_path="chromedriver.exe")
    
    record=[]
    url=get_url(search_item)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup=BeautifulSoup(driver.page_source,'html.parser')
        results =soup.find_all('div',{"data-component-type":"s-search-result"})
        
        for item in results:
            record =extract_record(item)         
            if record:
                records.append(record)
        
    driver.close()
        
    #save data as csv file
    with open('results.csv','w',newline='',encoding='utf-8')as f:
        writer=csv.writer(f)
        writer.writerow(['Description','Price','Rating','ReviewCount','url'])
        writer.writerows(records)
        
    
    

In [157]:
main('android phone')